# Allele-Specific Expression

RNA sequencing can distinguish transcripts expressed from different copies of genes on homologous chromosomes when single-nucleotide polymorphisms (perhaps silent) distinguish the two alleles. Linkage between these distinctive SNPs and _cis_-regulatory sequences can provide information on regulatory variation within a shared cellular context.

## Null Hypothesis Testing

The null hypothesis in allele-specific expression analysis is that the alleles are expressed equally and so each read is equally likely to be derived from each allele.

Here, we'll take two approaches to get a *p* value for the null hypothesis of equal expression in situations where just 25% of the reads come from one allele and 75% from the other. We'll look at this with just 8 reads, with 32 reads, and then with 100 reads.

### Permutation Testing

First, we'll generate many random sets of data according to the null model and look at the distribution of allele skew in these random data. Our approach to generating random sets of reads is simple: we choose randomly between `0` and `1`, and then count how often we choose `1` by summing the results of this random choice.

The `numpy.random` submodule of the `numpy` module has functions to generate (pseudo-)random numbers. To use this, we'll `import` it and use it like this:

```
import numpy as np
print(np.random.choice(['A', 'C', 'G', 'T']))
print(np.random.choice(['A', 'C', 'G', 'T'], 3))
print(np.random.choice([0, 1], 8))
```

The quick trick to *count* how many of our random 0-or-1 choices came up 1 is to sum the vector using the `sum()` function, like this:

```
sum(np.random.choice([0, 1], 8))
```

Now, we'll generate 10,000 random samples of 8 reads each, and tabulate how many random samples have zero, one, ..., eight reads from that sample.

First we'll create an empty counting array with 9 entries for 0 through 8 inclusive
```
allele_counts_8 = [0] * 9
```
Then we'll run a for loop 10,000 times. We don't actually use the loop varible inside the `for` loop, so by convention we'll just call it `_`.
```
for _ in range(0, 10000):
    ...
```

Next, we'll plot the distribution by importing the plotting module. We can directly plot our list, we don't need to turn it into a Pandas `Series` first.
```
import matplotlib.pyplot as plt
plt.plot(allele_counts_8)
```

We can use this to ask: in what fraction of random samples do you see 12.5% (one-eighth) or fewer `1` reads? What can you conclude from seeing this kind of skew in a sample of 8 reads?

To do this, we can add up 
```
allele_counts_8[0] + allele_counts_8[1]
```
or, to make it easier to read
```
sum(allele_counts_8[0:2])
```
and then divide by the total number of random trials we generated (which should be 10,000)
```
sum(allele_counts_8)
```

What if we wanted to consider a strong skew in *either* direction? That is, we're interested in situations where we had ≤12.5% or ≥87.5% frequence of `1` alleles?

We would need to consider 7 or 8 reads in addition to 0 or 1
```
sum(allele_counts_8[7:9])
```

We can repeat the same analysis, for 24 reads per random sample.

* Make an empty, 25-entry count array
* Loop over 10,000 random trials
    * Generate a random choice of 24 x 0 or 1
    * Sum them up
    * Add the count to the overall tally

Now we can test for a similar skew in our 24-read samples -- what are the odds of seeing 3 or fewer reads from one allele and all the rest from the other?

What about a stronger skew: just 0 or 1 reads from one allele and all the rest from the other?

## Random Variables

There is probably a small but non-zero odds of getting this strong skew in the 24-read sample -- but we would need to generate a lot of random samples in order to figure out exactly how small. Instead, we can treat the number of reads from the `1` allele as a random variable with a binomial distribution. This isn't always a fair description of biological data, but it's a reasonable starting point here. 

### Binomial Distribution

The scipy package contains a statistics module with a sub-module specific for the binomial distribution. We can get the probability
```
P( k successes out of N trials with probability p of success per trial )
```
using
```
binom.pmf(k, N, p)
```
"pmf" here stands for "probability mass function".

For instance, we can ask about exactly 2 "successes" out of 8 "trials" -- think of this as 2 reads from the `1` allele out of 8 reads counted in total. We can also ask abotu exactly 2 reads from the `1` allele out of 32 total.

To start, install the `scipy` package (you should need to do this just once, ever) and import the `binom` sub-module of the `scipy.stats` (in every notebook where you want to use it)

```
import sys
!{sys.executable} -m pip install scipy
```
Then, you can use the `binom.pmf()` function like
```
binom.pmf(2, 8, 0.5)
```

We can also test a range of different k values:
```
print(binom.pmf(range(0,4), 24, 0.5))
```
This makes it easy to sum up across many possibilities:
```
print(sum(binom.pmf(range(0,4), 24, 0.5)))
```


We can use this to compute a precise value for the very small probability of the 1-or-fewer skew in 24 reads.

We'd need to run a lot of simulations to find that p-value reliably!

We can also plot this distribution and compare it to our simulations.

Either we need to convert our probabilities into counts per 10,000 trials by multiplying them
```
plt.plot(binom.pmf(range(0,24), 24, 0.5) * 10000)
plt.plot(allele_counts_24)
```



Or, we need to convert our random trial counts into probabilities by converting them into a Pandas series and dividing the counts by the total number of trials
```
import pandas as pd
plt.plot(binom.pmf(range(0,24), 24, 0.5))
plt.plot(pd.Series(allele_counts_24) / sum(allele_counts_24))
```

The two lines are so close it's hard to tell them  apart, and we don't know which color corresponds to which data set. We can use some codes to specify the color and type of the lines. The version below uses a solid red line for the binomial value and a dashed blue line with points for the simulations
```
plt.plot(binom.pmf(range(0,24), 24, 0.5) * 10000, '.-r')
plt.plot(allele_counts_24, 'o--b')
```

## Maximum Likelihood Estimation

Instead of simply testing whether allele expression is even, we want to _estimate_ the relative skew in expression. To do this, we will start by making a graph where we consider all possible bias values, and then figure out the likelihood function P( 4 reads out of 32 | bias ). We'll use the `binom.pmf` function again, but now we'll consider many different values for the 3rd _p_ parameter instead of the first one.

This graph looks similar to others that we've made, but the axes are different -- we'll add x and y axis labels to emphasize this difference.

First make an array of possible _p_ values of 0, 0.01, 0.02, ..., 1.00
```
skew = np.arange(0,1,0.01)
```


Then, compute the *likelihood*, that is, the probability of generating the observed data (4 reads out of 32) from the mode (our specified value of *p*). Conveniently, we can run these over the whole array of values at once.
```
binom.pmf(4, 32, skew)
```

Now, we can plot these likelihoods, and then use the `plt.xlabel()` and `plt.ylabel()` functions to add axis labels.
```
plt.plot(skew, binom.pmf(4, 32, skew))
plt.xlabel('Allele skew')
plt.ylabel('P(4 reads / 32 total)')
```

As we discussed before, we often want to work with log likelihood functions. In fact, `binom` has the built-in ability to give us a log probability that will be numerically stable when the actual likelihood is a very tiny number, using the `logpmf` method.
```
plt.plot(skew, binom.logpmf(4, 32, skew))
```

In this plot, the likelihood function looks pretty flat around 0.25, but we might want to adjust the y-axis to focus on the region of high likelihood -- the default puts a lot of emphasis on parts of the plot where the likelihood is very small. The `plt.axis(x_min, x_max, y_min, y_max)` function does this: 
```
plt.axis([0.0, 1.0, -10, 0])
```

We need to find the point on the x-axis where the likelihood is maximized. We can probably guess that this will happen at 0.125, but we can use algorithms from Scipy to find the best likelihood. These methods are typically expressed in terms of _minimization_, and so we'll minimize the negative log likelihood which is equivalent to finding the maximum of the likelihood.

To do this, we define a function to compute the negative log likelihood, called `negloglik`, and then use `minimize_scalar` from `scipy.optimize` to find the allele skew value that maximizes the likelihood of our data.

This will import the minimization sub-module
```
from scipy.optimize import minimize_scalar
```
The negative log likelihood function is quite simple:
```
def negloglik(skew):
    return -binom.logpmf(4, 32, skew)
```
And we can then use `minimize_scalar` to find the best skew estimate
```
best = minimize_scalar(negloglik, bounds=(0,1), method='bounded')
```

To see how the total number of reads _N_ affects the sharpness of our maximum likelihood estimates, we'll plot the log likelihood functions for:
* 1 read out of 8
* 4 reads out of 32
* 16 reads out of 128